# SUSA CX Kaggle Capstone Project
## Part 3: Hyperparameter Tuning, Decision Trees, Ensemble Learning

### Table Of Contents
* [Introduction](#section1)
* [Deep Learning](#section2)
* [Final Kaggle Evaluation](#section3)   
* [Conclusion](#conclusion)
* [Additional Reading](#reading)


### Hosted by and maintained by the [Statistics Undergraduate Students Association (SUSA)](https://susa.berkeley.edu). Originally authored by [Patrick Chao](mailto:prc@berkeley.edu) & [Arun Ramamurthy](mailto:contact@arun.run).

<a id='section1'></a>
# SUSA CX Kaggle Capstone Project

Woohoo - you've made it to the end of the CX Kaggle Capstone Project! Congratulations on all of your hard work so far. We hope you've enjoyed this opportunity to learn new modeling techniques, some underlying mathematics, and even make new friends within your CX Kaggle team. At this point, we've covered the entirety of the Data Science Worflow, linear regression, feature engineering, PCA, shrinkage, hyperparameter tuning, decision trees and even ensemble models. This week, we're going to finish off this whirlwind tour with a revisit to our old friend, Deep Learning. While the MNIST digit dataset was really interesting to look at as a cool toy example of the powers of DL, this time you're going to apply neural networks to your housing dataset for some hands-on practice using Keras. 

> ### CX Kaggle Competition & Final Kaggle Evaluation
After you get some practice with deep learning, this week we will be asking you and your team to select and finalize your best model, giving you the codespace to write up your finalized model and evaluate it by officially submitting your results to Kaggle. The winners of this friendly collab-etition will be honored at the SUSA Banquet next Friday, including prizes for the winning team! We also want to encourage and facilitate discussion between teams on why different models performed differently, and give you a chance to chat with other teams about their own experiences with the CX Kaggle Capstone. 

## Logistics

Most of the logistics are the same as last week, but we are repeating them here for your convenience. Please let us know if you or your teammates are feeling nervous about the pace of this project - remember that we are not grading you on your project, and we really try to make the notebooks relatively easy and fast to code through. If for any reason you are feeling overwhelmed or frustrated, please DM us or talk to us in person. We want all of you to have a productive, healthy, and fun time learning data science! If you have any suggestions or recommendations on how to improve, please do not hesitate to reach out!

### Datathon

For those of you that came to the Datathon, we hope you all had a great time! It seemed that many people got a lot of work done and learned a ton of new materials. We had a quick lecture by Patrick on the mathematical intuitions on regularization and PCA. If you couldn't make it or didn't understand all the details, feel free to reach out! Also if you felt that the lecture was very helpful and you would like to see more similar stuff, let us know!

### Mandatory Office Hours

Because this is such a large project, you and your team will surely have to work on it outside of meetings. In order to get you guys to seek help from this project, we are making it **mandatory** for you and your group to attend **two (2)** SUSA Office Hours over the next 4 weeks. This will allow questions to be answered outside of the regular meetings and will help promote collaboration with more experienced SUSA members.

The schedule of SUSA office hours are below:
https://susa.berkeley.edu/calendar#officehours-table

We understand that most of you will end up going to Arun or Patrick's office hours, but we highly encourage you to go to other people's office hours as well. There are many qualified SUSA mentors who can help and this could be an opportunity for you to meet them.

In [1]:
# Import statements
from sklearn import tree # There are lots of other models from this module you can try!
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression, Ridge
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

sqrt=np.sqrt

/home/arun/.conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/arun/.conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


ModuleNotFoundError: No module named 'pydotplus'

<a id='section2'></a>
# Deep Learning

We may imagine hyperparameters as a bunch of individual knobs we may turn. Consider that we are visiting our friend and staying at her place. However, you did not realize that she is actually an alien and her house is filled with very strange objects. When you head to bed, you attempt to use her shower, but see that her shower is has a dozen of knobs that control the temperature of the water coming out! We only have a single output to work off of, but many different knobs or *parameters* to adjust. If the water is too hot, we can turn random knobs until it becomes cold, and learn a bit about our environment. We may determine that some knobs are more or less sensitive, just like hyperparameters. Each knob in the shower is equivalent to a hyperparameter we can tune in a model.

<a id='section3'></a>
# Final Kaggle Evaluation

Congrats on finishing the last of the models we planned on teaching you about during the CX Kaggle Capstone! 

You have now covered five distinct models and a several related techniques to add to your data science bag-of-tricks: 
- Linear Models
    - Multivariate Linear Regression
    - Polynomial Regression
    - Shrinkage / Biased Regression / Regularization (i.e. Ridge, LASSO)
- Decision Trees
    - Random Forests
- Deep Learning
    - Sequential Neural Networks
- Auxiliary Techniques 
    - The Data Science Workflow
    - Data Cleaning
    - Interpreting EDA Graphs
    - Feature Engineering
    - Principal Component Analysis
    - Hyperparameter Tuning (i.e. grid search)
    - Ensemble Learning (i.e. bagging, boosting) 
    
Wow, that's a lot! We are really proud of you all for exploring these techniques, which constitute some of Berkeley's toughest machine learning and statistics classes. As always, if you want to learn more about any of these topics, or are hungry to learn about even more techniques, feel free to reach out to any one of the SUSA Mentors.

With the help of the above listing and your own team's preferences, choose a model and a couple of techniques to implement for your final model. We will provide you with a preamble and some space to construct and train your model, as well as a helper function to turn your output into an official Kaggle submission file.  

In [6]:
################
### PREAMBLE ###
################
train = None
test = None 

####################
### MODEL DESIGN ###
####################
model = None 
# ^^ REPLACE THIS LINE ^^

################
### TRAINING ###
################

###############
### TESTING ###
###############
test_predictions = None
# ^^ REPLACE THIS LINE ^^

##################
### SUBMISSION ###
##################
def generate_kaggle_submission(predictions, test = test):
    '''
    This function accepts your 1459-dimensional vector of predicted SalesPrices for the test dataset, 
    and writes a CSV named kaggle_submission.csv containing your vector in a form suitable for 
    submission onto the Kaggle leaderboard.
    '''
    print("hello")
generate_kaggle_submission(test_predictions)

hello


As you might have noticed in the code block above, we had to write a simple CSV file containing row IDs and predicted values for the 1459 houses in the test dataset. This submission file is your ticket to 

Take a look at your `kaggle_submission.csv` file. When you and your team are ready, follow these instructions to upload your predictions to Kaggle and receive an official Kaggle score:

> TODO

# Conclusion

This brings us to an end to the CX Kaggle Capstone Project, as well as the Spring 2018 semester of SUSA Career Exploration. Congratulations on graduating from the SUSA Career Exploration committee! It's been a wonderful experience teaching you all, and we hope you got as much out of CX as we did this semester. This semester brought several new pilot programs to CX, such as the crash courses, workbooks, a revamped curriculum, and the CX Kaggle Capstone Project. You all have been great sources of feedback, and we want to make next semester's CX curriculum even better for the new generation of CX! 

We're going to ask you for feedback one last time, to give us insight into how we can improve the CX Kaggle Capstone experience for future CX members. Please fill out [this feedback form] and let us know how we could have done better. Thank you again for a wonderful semester, and we will see you again 



As always, please email [Arun Ramamurthy](mailto:contact@arun.run), [Patrick Chao](mailto:prc@berkeley.edu), or [Noah Gundotra](mailto:noah.gundotra@berkeley.edu) with any questions or concerns whatsoever. Have a great summer, and we hope to see you as a returning member in the Fall! Go SUSA!!!

With geom_love,
Lucas, Arun, Patrick, Noah, and the rest of the SUSA Board

<a id='reading'></a>
# Additional Reading

This notebook we covered a lot of different modeling tools. However, we have barely scratched the surface. Even though we have not presented as much math as in previous presentations, there is a lot of theory behind these tools. 

The most widely used tools are variants of decision trees. One of the most popular variants of random forest modeling is called [Gradient Boosting](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting). The framework that implements this particularly well is called [XGBoost](https://github.com/dmlc/xgboost).

Sci-kit learn has a lot of ensemble methods built-in as well. Their documentation is probably some of the best out there, it's absolutely littered with examples. See the [ensemble docs](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) for more. 

If you're interested in different types of models to use in your ensemble, try out Logistic Regression in the sci-kit learn library. For the full list of models, see [their docs](http://scikit-learn.org/stable/supervised_learning.html